In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# upload exported cities.csv file from WeatherPy.ipynb as dataframe
cities_df = pd.read_csv("../Output_data/cities.csv")
del cities_df["Unnamed: 0"]
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Bethel,90,US,1586444460,100,41.37,-73.41,52.00,5.82
1,Torbay,75,CA,1586444730,69,47.67,-52.73,37.99,21.92
2,Ushuaia,39,AR,1586444731,66,-54.80,-68.30,57.20,5.99
3,Altos del Rosario,100,CO,1586444732,65,8.79,-74.17,89.60,1.74
4,Bluff,98,NZ,1586444733,90,-46.60,168.33,54.00,7.00


In [3]:
# configure gmaps
fig = gmaps.figure()
gmaps.configure(api_key = g_key)

# drop NA values
cities_df = cities_df.dropna()

# set up variables
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

# create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 10, point_radius = .5)

fig.add_layer(heat_layer)

# display map
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# create new dataframe with ideal weather
ideal_df = cities_df

# max temp between 70 & 80 degrees, wind speed below 10 MPH, cloudiness = 0
ideal_df = ideal_df.loc[(ideal_df["Max Temp"] <= 80) & (ideal_df["Max Temp"] >= 70)]
ideal_df = ideal_df.loc[(ideal_df["Wind Speed"] < 10)]
ideal_df = ideal_df.loc[(ideal_df["Cloudiness"] == 0)]

# drop NA values and show header
ideal_df = ideal_df.dropna()
ideal_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
53,Jardim,0,BR,1586444787,42,-21.48,-56.14,79.48,3.83
199,Asău,0,RO,1586444949,20,46.43,26.40,73.40,3.36
203,Deleşti,0,RO,1586444953,20,46.70,27.55,73.99,3.36
332,Taşca,0,RO,1586445099,20,46.90,26.02,72.00,1.72
420,Sabha,0,LY,1586445200,18,27.04,14.43,76.12,9.80


In [5]:
# create hotel_df dataframe and add hotel column
hotel_df = pd.DataFrame(ideal_df)
hotel_df["Hotel"] = ""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel
53,Jardim,0,BR,1586444787,42,-21.48,-56.14,79.48,3.83,
199,Asău,0,RO,1586444949,20,46.43,26.40,73.40,3.36,
203,Deleşti,0,RO,1586444953,20,46.70,27.55,73.99,3.36,
332,Taşca,0,RO,1586445099,20,46.90,26.02,72.00,1.72,
420,Sabha,0,LY,1586445200,18,27.04,14.43,76.12,9.80,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
locations = hotel_df[["Lat", "Lng"]].astype(float)
params = {
    "location": locations,
    "rankby": "distance" <,
    "type": "hotel",
    "key": g_key,
}
